# Deploy a dask cluster with AWS cloudformation

## Objectives

Create a deploy a dask cluster using AWS cloud formation (success see file blabla). The current template creates:
- 1 dask-scheduler and 3 dask-workers. 
- A security group that allows a local machine to connect to the cluster (see the other notebook)
- EC2 role and connects this all instances to allow S3 access.
- S3 bucket in which we can store csv files

Things you have to do manually:
- upload the csv files to the s3 bucket
- create a user with programmatic access and configure the aws cli to work with these keys.

### TODO
- scheduler can be t2.micro but I want workers to be larger.
- use parameters to select the instance type for the scheduler and worker.
- To have 3 workers template of one is copied 3 times. It is better to maybe use auto scaling groups and define the number workers

### CloudFormation Template

execute the cell below if you want to see the entire cloud formation template in the notebook

In [ ]:
%load cf-dask-cluster.yaml

You can use this template to create a CloudFormation stack on AWS. 

Once the stack is completed and all EC2 instances are running you can take the public IP of the scheduler and copy this in the code snippet below.

Everything should work smoothly (for me it did)

In [1]:
from dask.distributed import Client

# copy the public ip of scheduler from the output section of cloudformation
ip = "35.153.178.80"
address = f"tcp://{ip}:8786"
dashboard = f"http://{ip}:8787/status"

print(f"Use the link below to connect to the cluster dashboard:\n{dashboard}")

print(address)
client = Client(address=address)

client

Use the link below to connect to the cluster dashboard:
http://35.153.178.80:8787/status
tcp://35.153.178.80:8786


C:\Users\sacha\anaconda3\envs\dask-cluster\lib\site-packages\distributed\client.py:1606: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | Client | Scheduler | Workers |
+-------------+--------+-----------+---------+
| cloudpickle | 3.0.0  | 3.1.0     | 3.1.0   |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://172.31.32.106:8786' processes=3 threads=6, memory=11.46 GiB>

In [2]:
import dask.array as da

a_da = da.ones(10, chunks=5)
a_da

dask.array<ones_like, shape=(10,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>

In [3]:
a_da_sum = a_da.sum()
a_da_sum

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [4]:
a_da_sum.compute()

np.float64(10.0)

In [5]:
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3000, 3000))
xd

dask.array<normal, shape=(30000, 30000), dtype=float64, chunksize=(3000, 3000), chunktype=numpy.ndarray>

In [6]:
%%time
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3000, 3000))
yd = xd.mean(axis=0)
yd.compute()

CPU times: total: 0 ns
Wall time: 6.93 s


array([ 9.99985273,  9.99891763,  9.99823302, ...,  9.99959002,
       10.00071476, 10.00053456])

In [7]:
!aws s3 ls


An error occurred (InvalidAccessKeyId) when calling the ListBuckets operation: The AWS Access Key Id you provided does not exist in our records.


In [8]:
import dask.dataframe as dd

# Read all CSV files from the root of the bucket
ddf = dd.read_csv("s3://dask-input-data/*.csv", 
                  dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool},
                  blocksize="25MB" )


ddf

PermissionError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: The AWS Access Key Id you provided does not exist in our records.

In [ ]:
%%time
len(ddf)

In [ ]:
%%time
ddf.head(2)

In [ ]:
%%time
result = ddf.DepDelay.max()
result.compute()

In [ ]:
%%time
len(ddf[~ddf.Cancelled])

In [ ]:
%%time
ddf[~ddf.Cancelled].groupby("Origin")["Origin"].count().compute()

In [ ]:
%%time
ddf.groupby("Origin").DepDelay.mean().compute()

In [ ]:
%%time
ddf.groupby("DayOfWeek").DepDelay.mean().idxmax().compute()

In [ ]:
client.close()